In [ ]:
from langchain.chat_models import init_chat_model 
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnableMap
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import time
from IPython.display import display, Markdown
load_dotenv()

True

In [7]:
model = init_chat_model(model="deepseek/deepseek-chat-v3-0324:free",model_provider="openai") # initializing chat model

In [4]:
print(model.invoke([{"role":"system" , "content": "what language do you think the user is speaking in?"} , {"role": "user", "content": "tussi ki kr rhe ho"}]).content) # low level model prompting

It looks like you're speaking **Punjabi**! The phrase **"tussi ki kr rhe ho"** translates to **"What are you doing?"** in English.  

Let me know if you'd like any help or translations! 


#### Runnables
Following I give examples of usage of various types of runnables:

- RunnableLambda: to wrap any python function into a runnable

In [5]:
def pr(text : str) -> str:
    print(f"Used a runnable to print {text}")

runnable = RunnableLambda(pr)
runnable.invoke("\"Hello world!\"")

Used a runnable to print "Hello world!"


- RunnableParallel: chain various tasks to execute in parallel

In [ ]:
def word_count_runnable(text : str) -> str:
    print(len(text.split()))

parallelRunnable = RunnableParallel(
    pr = pr , 
    word_count = word_count_runnable
)

parallelRunnable.invoke("Hello World! Testing 1 2 3 ....")

# Here both the funcitons were executed parallely and whatever they returned was retured as dict by the invoke call

Used a runnable to print Hello World! Testing 1 2 3 ....
7


{'pr': None, 'word_count': None}

- RunnableMap: same as RunnableParallel

In [7]:
runnable = RunnableMap(
    pr = pr,
    word_count = word_count_runnable
)

runnable.invoke("whats up")

Used a runnable to print whats up
2


{'pr': None, 'word_count': None}

- Chaining various runnables 

In [16]:
def add_three(num : int) -> int:
    return num + 3
def add_two(num : int) -> int:
    return num + 2
def add_one(num : int) -> int:
    return num + 1

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)
runnable_3 = RunnableLambda(add_three)

chain = runnable_1 | runnable_2 | runnable_3

print(chain.invoke(5))

11


#### Streaming : 
- Streaming is critical in making applications based on LLMs feel responsive to end-users.
- Basically here we print the tokens one by one as we get them instead of just printing then all together at the end to keep the user enagaged

In [ ]:
for i in model.stream([{"role":"system" , "content": "what language do you think the user is speaking in? and then write the phrase in the native script of the language"} , {"role": "user", "content": "tussi ki kr rhe ho"}]):
    print(i.content,end="")
    time.sleep(.1)

The user is speaking in **Punjabi**, written in the Latin script (Romanized Punjabi).  

Here’s the phrase in the native script (**Gurmukhi**):  
**ਤੁਸੀਂ ਕੀ ਕਰ ਰਹੇ ਹੋ?**  

*(Translation: "What are you doing?")*  

Let me know if you'd like help with anything else! 

#### Messages:
Here we can pass vraious types of mesasge to llm model to get better responses

In [15]:
message = [
    SystemMessage("You are a helpful ai chatbot whose task is to help the user in any way possible and be utmost polite with them"),
    HumanMessage("Hello how are you"),
    AIMessage("I am fine, how are you doing?"),
    HumanMessage("I am feeling sad today suggest something fun to me")
]

display(Markdown(model.invoke(message).content))

I'm so sorry to hear you're feeling sad today—I'm here to help brighten your mood! Here are some fun ideas to lift your spirits:  

✨ **Watch a Comfort Show/Movie**: Choose something light-hearted, funny, or nostalgic—maybe a favorite comedy or a feel-good animated film.  
🎵 **Dance Like Nobody’s Watching**: Put on your favorite upbeat songs and have a mini dance party. Movement releases endorphins!  
🎨 **Get Creative**: Doodle, paint, or try a simple craft. Even silly sketches can be therapeutic.  
🌿 **Take a Nature Break**: If possible, step outside for fresh air—watch clouds, listen to birds, or feel the sunshine.  
📖 **Read Something Uplifting**: A funny book, inspiring quotes, or happy short stories can help shift your mindset.  
🍿 **Treat Yourself**: Make your favorite snack or try a new recipe. Comfort food (in moderation) can be a mood booster!  
🎮 **Play a Silly Game**: Try a mobile game, puzzle, or even nostalgic childhood games for a quick distraction.  
😂 **Watch Comedy Sketches**: YouTube or streaming platforms have great stand-up clips or funny animal videos.  

Would you like something more specific—like music recommendations, a funny video link, or a short mindfulness exercise? You're not alone—I’m here to chat if you’d like! 💛

#### Prompt Template:
- Most of the times the prompt we give to the ai is based on a tmeplate with multiple vairables or specific structure.
-  We can use this runnable to best manage messages

In [46]:
systemTemplate = "You are a helpful assitant chatbot to assist the user in any way possible"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , systemTemplate) , 
        ("user" , "{text}")
    ]
)

chain = LLMChain(prompt = prompt , llm = model)

for i in chain.stream("write 10 pointers on equality"): # sop this shows that we can't easily stream the output in a chain
    print(i['text'] , end="",flush=True)

Here are 10 key pointers on **equality**:

1. **Equal Rights** – Every individual should have the same legal, social, and political rights, regardless of gender, race, religion, or economic status.  
2. **Non-Discrimination** – No person should face prejudice or bias in education, employment, or public services based on identity.  
3. **Equal Opportunities** – Everyone deserves fair access to jobs, education, healthcare, and resources to succeed.  
4. **Gender Equality** – Men and women should have equal rights, pay, and representation in leadership roles.  
5. **Social Justice** – Systems and policies must address historical inequalities to create a fairer society.  
6. **Economic Equality** – Reducing wealth gaps ensures balanced growth and opportunities for all economic classes.  
7. **Inclusion** – Diversity should be embraced, ensuring marginalized groups (LGBTQ+, disabled, minorities) are represented.  
8. **Equal Before the Law** – Justice systems must treat all individuals fair

In [42]:
for i in model.stream(prompt.invoke("write 10 pointers on equality")):
    print(i.content, end="",flush=True)

Certainly! Here are 10 key pointers on **equality**:  

1. **Equal Rights** – Everyone deserves the same fundamental rights and freedoms, regardless of gender, race, religion, or background.  

2. **No Discrimination** – Equality means treating people fairly without bias or prejudice based on identity, caste, or economic status.  

3. **Equal Opportunities** – Access to education, jobs, and healthcare should be available to all without unfair barriers.  

4. **Gender Equality** – Equal treatment and representation for all genders in workplaces, leadership, and households.  

5. **Legal Equality** – The law should protect all individuals equally and ensure justice without favoritism.  

6. **Intersectionality** – Recognizing that discrimination can overlap (e.g., race + gender + class) strengthens efforts toward true equality.  

7. **Fair Pay & Representation** – Equal pay for equal work and proportional representation in decision-making bodies.  

8. **Social Inclusion** – Every indiv